In [18]:
import requests
import pickle
from flask import Flask, request, jsonify

# QUESTION 1

In [42]:
# pip install pipenv
# $ pipenv --version
# pipenv, version 2024.2.0

# QUESTION 2

In [40]:
# pipenv install scikit-learn==1.5.2 flask gunicorn

In [43]:
# "scikit-learn": {
            # "hashes": [
            #     "sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445",

In [10]:
PREFIX = "https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework"
files = {
    "model": f"{PREFIX}/model1.bin",
    "dv": f"{PREFIX}/dv.bin"
}

for file_name, url in files.items():
    response = requests.get(url)
    response.raise_for_status()
    with open(f"{file_name}.bin", "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_name}.bin")

Downloaded model.bin
Downloaded dv.bin


# QUESTION 3

In [11]:
with open("model.bin", "rb") as model_file, open("dv.bin", "rb") as dv_file:
    model = pickle.load(model_file)
    dv = pickle.load(dv_file)

c:\Users\ctw02071\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ctw02071\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
client = {
    "job": "management",
    "duration": 400,
    "poutcome": "success"
}

In [13]:
X = dv.transform([client])

prediction = model.predict_proba(X)[0, 1]

print(f"The probability that the client will get a subscription is: {prediction:.3f}")

The probability that the client will get a subscription is: 0.909


# QUESTION 4

In [44]:
## predict.py ##

# import pickle
# from flask import Flask, request, jsonify

# # PREFIX = "https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework"
# # files = {
# #     "model": f"{PREFIX}/model1.bin",
# #     "dv": f"{PREFIX}/dv.bin"
# # }

# # for file_name, url in files.items():
# #     response = requests.get(url)
# #     response.raise_for_status()
# #     with open(f"{file_name}.bin", "wb") as f:
# #         f.write(response.content)
# #     print(f"Downloaded {file_name}.bin")

# model = "model.bin"
# dv = "dv.bin"

# with open(model, "rb") as model_file, open(dv, "rb") as dv_file:
#     model = pickle.load(model_file)
#     dv = pickle.load(dv_file)
    
# app = Flask('subscription')

# @app.route('/predict', methods=['POST'])
# def predict():
#     client = request.get_json()
#     X = dv.transform([client])
#     y_pred = model.predict_proba(X)[0, 1]  
    
#     result = {
#         'subscription-probability': float(y_pred),
#     }
    
#     return jsonify(result)
    
    
# if __name__ == '__main__':
#     app.run(debug=True, host='localhost', port=9698)

In [ ]:
#pipenv run gunicorn --bind 0.0.0.0:9698 predict:app

In [31]:
url = 'http://localhost:9698/predict'

In [32]:
client = {"job": "student", "duration": 280, "poutcome": "failure"}

In [33]:
response = requests.post(url, json=client).json()
response

{'subscription-probability': 0.33480703475511053}

# QUESTION 5

In [28]:
#$ docker pull svizor/zoomcamp-model:3.11.5-slim

In [29]:
#$ docker images
#svizor/zoomcamp-model   3.11.5-slim    975e7bdca086   8 days ago     130MB

In [30]:
## DOCKERFILE ##

# FROM svizor/zoomcamp-model:3.11.5-slim

# RUN pip install pipenv

# WORKDIR /app

# COPY ["Pipfile", "Pipfile.lock", "./"]

# RUN pipenv install --system --deploy

# COPY ["predict.py", "model.bin", "dv.bin", "./"]

# EXPOSE 9698

# ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9698", "predict:app"]

In [34]:
# $ docker build -t zoomcamp-05-deployment .

# QUESTION 6

In [35]:
#$ docker run -it --rm -p 9698:9698 zoomcamp-05-deployment

In [38]:
client = {"job": "management", "duration": 400, "poutcome": "success"}

In [39]:
response = requests.post(url, json=client).json()
response

{'subscription-probability': 0.7590966516879658}